# Customer Funnel Analysis

Dataset : https://www.kaggle.com/aerodinamicc/ecommerce-website-funnel-analysis

In [ ]:
import pandas as pd

In [ ]:
folder = "ecommerce_website"
df_homepage = pd.read_csv(f"{folder}/home_page_table.csv")
df_payment_confirm = pd.read_csv(f"{folder}/payment_confirmation_table.csv")
df_payment = pd.read_csv(f"{folder}/payment_page_table.csv")
df_search = pd.read_csv(f"{folder}/search_page_table.csv")

In [ ]:
df_user = pd.read_csv(f"{folder}/user_table.csv",parse_dates = ['date'])

# Pemahaman data

In [ ]:
df_user.sample(2)

In [ ]:
print(df_user.shape)
df_user['user_id'].nunique()

In [ ]:
display(df_homepage.sample(2))
df_homepage['page'].value_counts()

In [ ]:
display(df_search.sample(2))
df_search['page'].value_counts()

In [ ]:
display(df_payment.sample(2))
df_payment['page'].value_counts()

In [ ]:
display(df_payment_confirm.sample(2))
df_payment_confirm['page'].value_counts()

## Kesimpulan awal :   
Data dapat digabung

## Preprocessing data agar mudah diolah

In [ ]:
df_user_activity = pd.concat([df_homepage,df_search,df_payment,df_payment_confirm],axis=0)
df_user_activity['value'] = 1
df_user_activity

In [ ]:
df_user_activity_pivot = df_user_activity.pivot(index='user_id',columns='page',values='value')
df_user_activity_pivot

In [ ]:
funnel_cols = ['home_page','search_page','payment_page','payment_confirmation_page']

df_user_activity_pivot = df_user_activity_pivot.fillna(0).reset_index()
df_user_activity_pivot = df_user_activity_pivot[['user_id']+funnel_cols]
df_user_activity_pivot

In [ ]:
df_funnel = df_user.merge(df_user_activity_pivot,on=['user_id'],how='inner')
df_funnel

# Funnel Analysis

In [ ]:
pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
df_plot_funnel = df_funnel[funnel_cols].sum(axis=0).to_frame('count').reset_index()
df_plot_funnel

In [ ]:
df_funnel.sort_values('date')

In [ ]:
df_plot_funnel = df_funnel[funnel_cols].sum(axis=0).to_frame('count').reset_index()
fig = px.funnel(df_plot_funnel,x='index',y='count')
fig.show()

In [ ]:
df_plot_funnel = df_funnel.query("date<'2015-02-01'")[funnel_cols].sum(axis=0).to_frame('count').reset_index()
fig = px.funnel(df_plot_funnel,x='index',y='count')
fig.show()

In [ ]:
df_plot_funnel = df_funnel.query("date>='2015-04-01'")[funnel_cols].sum(axis=0).to_frame('count').reset_index()
fig = px.funnel(df_plot_funnel,x='index',y='count')
fig.show()

In [ ]:
df_plot_funnel = df_funnel[funnel_cols].sum(axis=0).to_frame('count').reset_index()
df_plot_funnel['count_from_previous'] = df_plot_funnel['count'].shift(1)

In [ ]:
df_plot_funnel['conversion_rate'] = df_plot_funnel['count']/df_plot_funnel['count_from_previous']
df_plot_funnel

# Trend Analysis

In [ ]:
df_funnel

In [ ]:
df_funnel.groupby('date')[funnel_cols].sum().plot()

In [ ]:
df_funnel_daily = df_funnel.groupby(['date','sex','device'],as_index=False)[funnel_cols].sum()
df_funnel_daily

In [ ]:
df_funnel_daily = df_funnel_daily.melt(id_vars=['date','sex','device'],value_vars = funnel_cols,var_name='funnel',value_name='count')
df_funnel_daily

In [ ]:
import seaborn as sns

g = sns.FacetGrid(df_funnel_daily, col="device",  row="sex")
g.map(sns.lineplot, "date", "count","funnel")

In [ ]:
df_funnel_daily = df_funnel.groupby(['date','device'],as_index=False)[funnel_cols].sum()
df_funnel_daily = df_funnel_daily.melt(id_vars=['date','device'],value_vars = funnel_cols,var_name='funnel',value_name='count')
df_funnel_daily
import seaborn as sns
import matplotlib.pyplot as plt


g = sns.FacetGrid(df_funnel_daily, col="device",height=10)
g.map(sns.lineplot, "date", "count","funnel")
